# Smooth Monotonic Networks: Experiments on UCI bechmark functions
## General definitions

Among others, we compare against XGBoost, which can be installed via `pip install xgboost`, and 
the Hierarchical Lattice Layer, which can be installed via 
`pip install pmlayer`.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd

import random

import torch 
import torch.nn as nn

from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import r2_score as r2
from sklearn.isotonic import IsotonicRegression
from sklearn.model_selection import KFold

from scipy.stats import wilcoxon

import matplotlib.pyplot as plt
from tqdm.notebook import tnrange

from xgboost import XGBRegressor

from pmlayer.torch.layers import HLattice

from MonotonicNN import SmoothMonotonicNN, SmoothMonotonicNNAlt, MonotonicNN, SMM_MLP
from MonotonicNNPaperUtils import Progress, MLP, total_params, fit_torch, fit_torch_val

from sklearn.model_selection import train_test_split

from sklearn import preprocessing


In [3]:
n_folds = 5  # number of splits in cross-validation
n_trees = 100  # number of trees for XGBoost
K = 6  # number of SMM groups and neurons per group
k = 3  # lattice points per dimension with constraint
transform = "exp"  # way positivity is ensured in SMM
mlp_neurons = 64
maxiter = 100000  # upper bound on epochs, never reached for this value

prefix = "supp-"

## Cross-validation
The function doing the cross-validation, relying on global variables for all hyperparameters.

In [4]:
def do_the_cv():
    seed = 42
    clip = True
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    
    no_methods = 4
    kf = KFold(n_splits=n_folds)
    mse_train=np.zeros((no_methods, n_folds))
    mse_test=np.zeros((no_methods, n_folds))
    no_params=np.zeros(no_methods)
    
    for split, (train, test) in enumerate(kf.split(x)):
        x_train, x_val, y_train, y_val = train_test_split(x[train], y[train], test_size=0.25)
        x_test = x[test]
        y_test = y[test]
        
        #print(y_train.shape, y_val.shape, y_test.shape,)
        
        m = 0
        print("split:", split)
        x_train_torch = torch.from_numpy(x_train.astype(np.float32)).clone()
        x_test_torch = torch.from_numpy(x_test.astype(np.float32)).clone()
        x_val_torch = torch.from_numpy(x_val.astype(np.float32)).clone()
        y_train_torch = torch.from_numpy(y_train.astype(np.float32)).clone()
        y_test_torch = torch.from_numpy(y_test.astype(np.float32)).clone()
        y_val_torch = torch.from_numpy(y_val.astype(np.float32)).clone()
        
        print("SMM-MLP: ", end='')
        model = SMM_MLP(dim, increasing, mlp_neurons, K, transform)
        model_best = SMM_MLP(dim, increasing, mlp_neurons, K, transform)
        fit_torch_val(model, model_best, 
                      x_train_torch, y_train_torch, 
                      x_val_torch, y_val_torch, 
                      max_iterations=maxiter)
        y_smm_train = model_best(x_train_torch).detach().numpy().reshape(-1,1)
        y_smm_test = model_best(x_test_torch).detach().numpy().reshape(-1,1)
        mse_train[m, split] = mse(y_train, y_smm_train)
        if not clip:
            mse_test[m, split] = mse(y_test, y_smm_test)
        else:
            mse_test[m, split] = mse(y_test, np.clip(y_smm_test, 0., 1.))
        print(mse_test[m, split])
        if(split == 0):
            no_params[m] = total_params(model)
        m=m+1
        
        print("SMM: ", end='')
        model = SmoothMonotonicNNAlt(dim, K, K, beta=-1., mask=mask, transform=transform)
        model_best = SmoothMonotonicNNAlt(dim, K, K, beta=-1., mask=mask, transform=transform)
        fit_torch_val(model, model_best, 
                      x_train_torch, y_train_torch, 
                      x_val_torch, y_val_torch, 
                      max_iterations=maxiter)
        y_smm_train = model_best(x_train_torch).detach().numpy().reshape(-1,1)
        y_smm_test = model_best(x_test_torch).detach().numpy().reshape(-1,1)
        mse_train[m, split] = mse(y_train, y_smm_train)
        if not clip:
            mse_test[m, split] = mse(y_test, y_smm_test)
        else:
            mse_test[m, split] = mse(y_test, np.clip(y_smm_test, 0., 1.))
        print(mse_test[m, split])
        if(split == 0):
            no_params[m] = total_params(model)
        m=m+1  

        print("XG: ", end='')
        model = XGBRegressor(monotone_constraints=xg_constraints, n_estimators=n_trees, 
                             early_stopping_rounds=(n_trees // 10), verbosity=0)
        model.fit(x_train, y_train, eval_set=[(x_train, y_train), (x_val, y_val)], verbose=0)           
        y_xg_train = model.predict(x_train)
        y_xg_test = model.predict(x_test)
        mse_train[m, split] = mse(y_train, y_xg_train)
        if not clip:
            mse_test[m, split] = mse(y_test, y_xg_test)
        else:
            mse_test[m, split] = mse(y_test, np.clip(y_xg_test, 0., 1.))
        print(mse_test[m, split])
        if(split == 0):
            no_params[m] = n_trees
        m=m+1

        print("HLL: ", end='')
        input_len = dim - len(increasing)
        output_len = torch.prod(lattice_sizes_tensor).item()
        ann = MLP(input_len, output_len, mlp_neurons)
        model = HLattice(dim, lattice_sizes_tensor, increasing, ann)
        ann_best = MLP(input_len, output_len, mlp_neurons)
        model_best = HLattice(dim, lattice_sizes_tensor, increasing, ann_best)
        fit_torch_val(model, model_best, 
                      x_train_torch, y_train_torch, 
                      x_val_torch, y_val_torch, 
                      max_iterations=maxiter)
        y_hll_train = model_best(x_train_torch).detach().numpy().reshape(-1,1)
        y_hll_test = model_best(x_test_torch).detach().numpy().reshape(-1,1)
        mse_train[m, split] = mse(y_train, y_hll_train)
        if not clip:
            mse_test[m, split] = mse(y_test, y_hll_test)
        else:
            mse_test[m, split] = mse(y_test, np.clip(y_hll_test, 0., 1.))                           
        print(mse_test[m, split])
        if(split == 0):
            no_params[m] = total_params(model)
        m=m+1
        
    return mse_train, mse_test, no_params
        

## Energy

In [5]:
df = pd.read_excel('https://archive.ics.uci.edu/ml/machine-learning-databases/00242/ENB2012_data.xlsx')

Some of the methods - not the SMM - require sependent and indepedent variables to be in [0,1]. To ensure that all test data points are in that interval, we normalize the data before we split into trainign and test sets. In a real-world setting, one of course does not have access to the test data for computing the normalization.

In [6]:
dim = 8  # number of input variables
x = df[["X1", "X2", "X3", "X4", "X5", "X6", "X7", "X8"]].values
y = df[["Y1"]]
scaler = preprocessing.MinMaxScaler()
x = scaler.fit_transform(x)
y = scaler.fit_transform(y)

increasing = (2,4,6)
mask = np.array([0,0,1,0,1,0,1,0])
xg_constraints=(0,0,1,0,1,0,1,0)
lattice_sizes = list(np.ones(len(increasing))*k)
lattice_sizes_tensor = torch.tensor(lattice_sizes, dtype=torch.long)

MSE_train, MSE_test, no_params = do_the_cv()
print(np.mean(MSE_test, axis=1))
print(np.median(MSE_test, axis=1))

split: 0
SMM-MLP: (1390) 0.006322825609784933
SMM: (2576) 0.006399342562287919
XG: 0.00985521379627735
HLL: (360) 0.008799495335652366
split: 1
SMM-MLP: (503) 0.0007289738972680103
SMM: (1523) 0.0002796277969128438
XG: 0.00019135722714272674
HLL: (317) 0.0008458875079661793
split: 2
SMM-MLP: (679) 0.0002101250256614929
SMM: (196) 0.005362573252940356
XG: 0.00024007329687893977
HLL: (364) 0.000899618185628404
split: 3
SMM-MLP: (1229) 0.0007658443331479123
SMM: (380) 0.005375499588880487
XG: 0.000244311695494322
HLL: (426) 0.0008613082937738197
split: 4
SMM-MLP: (1451) 0.0010170388845573803
SMM: (2044) 0.0015206217933186454
XG: 0.0002926511889826994
HLL: (217) 0.0027145926572363405
[0.00180896 0.00378753 0.00216472 0.00282418]
[0.00076584 0.00536257 0.00024431 0.00089962]


In [7]:
fn = prefix + "energy-y1-results-val.npz"
np.savez(fn, MSE_train=MSE_train, MSE_test=MSE_test, no_params= no_params)

In [8]:
y = df[["Y2"]]
y = scaler.fit_transform(y)

MSE_train, MSE_test, no_params = do_the_cv()
print(np.mean(MSE_test, axis=1))
print(np.median(MSE_test, axis=1))

split: 0
SMM-MLP: (674) 0.005242739974933524
SMM: (5377) 0.0027816216715962957
XG: 0.0029276042750775283
HLL: (253) 0.004589734663585562
split: 1
SMM-MLP: (305) 0.0034412554959758522
SMM: (317) 0.008286796736354332
XG: 0.00037576042753442203
HLL: (288) 0.0010379572126126078
split: 2
SMM-MLP: (1155) 0.0004250116053859394
SMM: (188) 0.007218048795356558
XG: 0.0003391859600550182
HLL: (595) 0.0008963351224246367
split: 3
SMM-MLP: (733) 0.001480684481291354
SMM: (427) 0.006485038618182394
XG: 0.0004159003832509222
HLL: (209) 0.0016643212670091995
split: 4
SMM-MLP: (896) 0.0008217619874593873
SMM: (260) 0.006701604460316123
XG: 0.0006519792903015522
HLL: (1075) 0.00443963815213139
[0.00228229 0.00629462 0.00094209 0.0025256 ]
[0.00148068 0.0067016  0.0004159  0.00166432]


In [9]:
fn = prefix + "energy-y2-results-val.npz"
np.savez(fn, MSE_train=MSE_train, MSE_test=MSE_test, no_params=no_params)

## Qsar

In [10]:
#data = np.genfromtxt("qsar_fish_toxicity.csv", delimiter=';')
#scaler = preprocessing.MinMaxScaler()
#data = scaler.fit_transform(data)
#x=data[:,0:-1]
#y=data[:,-1].reshape(-1,1)

In [11]:
df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/00504/qsar_fish_toxicity.csv', sep=';', header=None)
x = df[df.columns[0:6]].values
y = df[df.columns[6]].values
x = scaler.fit_transform(x)
y = scaler.fit_transform(y.reshape(-1, 1))

In [12]:
dim = 6
increasing = (1,5)
mask = np.array([0,1,0,0,0,1])
xg_constraints = (0,1,0,0,0,1)
lattice_sizes = list(np.ones(len(increasing))*k)
lattice_sizes_tensor = torch.tensor(lattice_sizes, dtype=torch.long)

MSE_train, MSE_test, no_params = do_the_cv()
print(np.mean(MSE_test, axis=1))
print(np.median(MSE_test, axis=1))

split: 0
SMM-MLP: (243) 0.009458758897124064
SMM: (266) 0.00979062954754857
XG: 0.008789900491549221
HLL: (127) 0.009585288023288705
split: 1
SMM-MLP: (251) 0.011999463597506163
SMM: (340) 0.01238168337489465
XG: 0.012767969794619896
HLL: (141) 0.01284292489695026
split: 2
SMM-MLP: (402) 0.01065847392323522
SMM: (133) 0.012332334836972464
XG: 0.010086175093449468
HLL: (169) 0.010687577621917395
split: 3
SMM-MLP: (276) 0.008075265393856697
SMM: (270) 0.00781132696199282
XG: 0.008286133279091383
HLL: (168) 0.008492504538430161
split: 4
SMM-MLP: (151) 0.007076661979225905
SMM: (208) 0.008057534331953685
XG: 0.007661475215480275
HLL: (122) 0.0075969402490896195
[0.00945372 0.0100747  0.00951833 0.00984105]
[0.00945876 0.00979063 0.0087899  0.00958529]


In [13]:
fn = prefix + "qsar-results-val.npz"
np.savez(fn, MSE_train=MSE_train, MSE_test=MSE_test, no_params=no_params)

## Concrete

In [14]:
df = pd.read_excel("https://archive.ics.uci.edu/ml/machine-learning-databases/concrete/compressive/Concrete_Data.xls")
print(df.columns[0:8])
print(df.columns[8])

Index(['Cement (component 1)(kg in a m^3 mixture)',
       'Blast Furnace Slag (component 2)(kg in a m^3 mixture)',
       'Fly Ash (component 3)(kg in a m^3 mixture)',
       'Water  (component 4)(kg in a m^3 mixture)',
       'Superplasticizer (component 5)(kg in a m^3 mixture)',
       'Coarse Aggregate  (component 6)(kg in a m^3 mixture)',
       'Fine Aggregate (component 7)(kg in a m^3 mixture)', 'Age (day)'],
      dtype='object')
Concrete compressive strength(MPa, megapascals) 


In [15]:
k = 3
dim = 8
x = df[df.columns[0:8]].values
y = df[df.columns[8]].values
scaler = preprocessing.MinMaxScaler()
x = scaler.fit_transform(x)
y = scaler.fit_transform(y.reshape(-1, 1))

In [16]:
increasing = (3,)
mask = np.array([0,0,0,1,0,0,0,0])
xg_constraints=(0,0,0,1,0,0,0,0)
lattice_sizes = list(np.ones(len(increasing))*k)
lattice_sizes_tensor = torch.tensor(lattice_sizes, dtype=torch.long)

MSE_train, MSE_test, no_params = do_the_cv()
print(np.mean(MSE_test, axis=1))
print(np.median(MSE_test, axis=1))

fn = prefix + "concrete-results-val.npz"
np.savez(fn, MSE_train=MSE_train, MSE_test=MSE_test, no_params=no_params)

split: 0
SMM-MLP: (411) 0.028815380894543385
SMM: (1391) 0.030746577001408613
XG: 0.011038243085424728
HLL: (374) 0.030034497449100316
split: 1
SMM-MLP: (438) 0.015019083414999352
SMM: (1153) 0.011679465507648246
XG: 0.016107357881940378
HLL: (857) 0.01500448106729225
split: 2
SMM-MLP: (368) 0.019475657294044317
SMM: (1554) 0.017200010454874032
XG: 0.008373064209947687
HLL: (345) 0.011380587482070516
split: 3
SMM-MLP: (430) 0.012195022131331078
SMM: (944) 0.005461114592146443
XG: 0.005664596166736345
HLL: (284) 0.006366697955732752
split: 4
SMM-MLP: (356) 0.036354975957647646
SMM: (753) 0.03699965426267193
XG: 0.05695978483048139
HLL: (193) 0.04573500110251762
[0.02237202 0.02041736 0.01962861 0.02170425]
[0.01947566 0.01720001 0.01103824 0.01500448]
